In [1]:
from PyQt5 import QtWidgets, uic, QtGui, QtMultimedia
from PyQt5.QtWidgets import (QMainWindow, QTextEdit,
                QAction, QFileDialog, QApplication)
from PyQt5.QtCore import QThread, pyqtSignal, QUrl

import mainwindow
import sys
import json
from os.path import join, dirname
from ibm_watson import SpeechToTextV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator


In [2]:
class CallWatson(QThread):
    throw_results_recognition = pyqtSignal(list, float)
    throw_results_translation = pyqtSignal(list)
    
    def __init__(self, file):
        QThread.__init__(self)
        self.path = file
        self.text = []
    
    def __del__(self):
        self.wait()
        
    def run_recognition(self):
        authenticator = IAMAuthenticator('ldfzprXgznF0Ti5cGYrYHRcCTJ57HC5Pzlvt2huCqQ39')
        speech_to_text = SpeechToTextV1(authenticator=authenticator)
        speech_to_text.set_service_url(
            'https://api.eu-gb.speech-to-text.watson.cloud.ibm.com/instances/011afe6f-1a89-4893-9443-4ab7558f05a6'
            )
        with open(self.path, 'rb') as audio_file:
            speech_recognition_results = speech_to_text.recognize(
                audio=audio_file,
                content_type='audio/mp3',
                speaker_labels=True
            ).get_result()
        recognized_words = speech_recognition_results['results'][0]['alternatives'][0]['timestamps']
        recognized_speakers = speech_recognition_results['speaker_labels']
        text = []
        ind_in_text = -1
        prev_speaker = -1
        for ind_in_json in range(len(recognized_words)):
            speaker = recognized_speakers[ind_in_json]['speaker']
            if speaker != prev_speaker:
                ind_in_text += 1
                text.append('Speaker ' + str(speaker + 1) + ':')
            text[ind_in_text] += ' ' + recognized_words[ind_in_json][0]
            prev_speaker = speaker
        
        self.text = text 
        self.throw_results_recognition.emit(
            text, speech_recognition_results['results'][0]['alternatives'][0]['confidence']
            )

        
    def run_translation(self):
        authenticator = IAMAuthenticator('jx6Q0eYZWzdZ8N9VcJOXmpubjt7hVpq5yHi-15vsT9sx')
        language_translator = LanguageTranslatorV3(
            version='2018-05-01',
            authenticator=authenticator
            )
        language_translator.set_service_url(
           'https://api.eu-gb.language-translator.watson.cloud.ibm.com/instances/07b13a19-da91-434a-90e7-f1957b4f3829'
            )
 
        translation = language_translator.translate(
            text=self.text,
            model_id='en-ru'
            ).get_result()
        russian_text = []
        for ind in range(len(translation['translations'])):
            russian_text.append(translation['translations'][ind]['translation'])
        self.throw_results_translation.emit(russian_text)

In [3]:
class MainWindow(QMainWindow):
    start_recognition = pyqtSignal()
    start_translation = pyqtSignal()
    
    def __init__(self):
        super(MainWindow, self).__init__()
        self.ui = mainwindow.Ui_MainWindow()
        self.ui.setupUi(self)
        self.ui.get_file_button.clicked.connect(self.get_file_name)
        self.ui.recognize_button.clicked.connect(self.call_recognition)
        self.ui.translate_button.clicked.connect(self.call_translation)
        self.ui.recognized_text.setVisible(False)
        self.ui.translated_text.setVisible(False)
        #self.ui.play_button.clicked.connect(self.play_audio)
        
        self.path = ""
        
    def get_file_name(self):
        self.path = QFileDialog.getOpenFileName(self, "Open Dialog", "", "*.mp3")[0]
    
    def catch_results_recognition(self,result, confidence):
        self.ui.get_file_button.setEnabled(True)
        self.ui.recognize_button.setEnabled(True)
        self.ui.translate_button.setEnabled(True)
        self.ui.recognized_text.setVisible(True)
        for line in result:
            self.ui.recognized_text.insertPlainText(line)
        
    def catch_results_translation(self,result):
        self.ui.get_file_button.setEnabled(True)
        self.ui.recognize_button.setEnabled(True)
        self.ui.translate_button.setEnabled(True)
        self.ui.translated_text.setVisible(True)
        for line in result:
            self.ui.translated_text.insertPlainText(line)
    
    def call_recognition(self):
        self.ui.get_file_button.setEnabled(False)
        self.ui.recognize_button.setEnabled(False)
        self.ui.translate_button.setEnabled(False)
        self.watson = CallWatson(self.path)
        self.start_recognition.connect(self.watson.run_recognition)
        self.watson.throw_results_recognition.connect(
            self.catch_results_recognition
            )
        self.start_recognition.emit()
    
    def call_translation(self):
        self.ui.get_file_button.setEnabled(False)
        self.ui.recognize_button.setEnabled(False)
        self.ui.translate_button.setEnabled(False)
        self.start_translation.connect(self.watson.run_translation)
        self.watson.throw_results_translation.connect(
            self.catch_results_translation
            )
        self.start_translation.emit()
    
    def play_audio(self):
        print("come to play")
        url = QUrl.fromLocalFile(self.path)
        content = QtMultimedia.QMediaContent(url)
        player = QtMultimedia.QMediaPlayer()
        player.setMedia(content)
        player.play()
        print("played")

In [ ]:
if __name__ == "__main__":
    app = QtWidgets.QApplication([])
    application = MainWindow()
    application.show()
    sys.exit(app.exec())